# Super Trend Stragety [Supertrend + EMA(200) + RSI(50) + ATR(14)] - v1

## Library 

In [ ]:
from smartapi import SmartConnect
import pandas as pd
import pandas_ta as ta
from datetime import datetime, timedelta
import credentials
import stock
import requests
import numpy as np
import json
from time import time, sleep
from talib.abstract import *
import threading
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#df = pd.read_excel("list.xlsx")
#SYMBOL_LIST = df['Symbol']

## Symbols List

In [ ]:
SYMBOL_LIST = ['BHEL,INFY,BHEL']
TRADED_SYMBOL = []
timeFrame = 60 + 5 #5 sec coz dealy repsone of historical API

## Order Place NSE Exchange 

In [ ]:
def place_order(token,symbol,qty,buy_sell,ordertype,price,variety= 'NORMAL',exch_seg='NSE',triggerprice=0):
    try:
        orderparams = {
            "variety": variety,
            "tradingsymbol": symbol,
            "symboltoken": token,
            "transactiontype": buy_sell,
            "exchange": exch_seg,
            "ordertype": ordertype,
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": price,
            "squareoff": "0",
            "stoploss": "0",
            "quantity": qty,
            "triggerprice":triggerprice
            }
        orderId=credentials.SMART_API_OBJ.placeOrder(orderparams)
        print("The order id is: {}".format(orderId))
    except Exception as e:
        print("Order placement failed: {}".format(e.message))

## Geting The Stock Details From Angel API

In [ ]:
def intializeSymbolTokenMap():
    url = 'https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json'
    d = requests.get(url).json()
    global token_df
    token_df = pd.DataFrame.from_dict(d)
    token_df['expiry'] = pd.to_datetime(token_df['expiry'])
    token_df = token_df.astype({'strike': float})
    credentials.TOKEN_MAP = token_df

## To Get The Stock Symbol and Token From API

In [ ]:
def getTokenInfo (symbol, exch_seg ='NSE',instrumenttype='OPTIDX',strike_price = '',pe_ce = 'CE'):
    df = credentials.TOKEN_MAP
    strike_price = strike_price*100
    if exch_seg == 'NSE':
        eq_df = df[(df['exch_seg'] == 'NSE') & (df['symbol'].str.contains('EQ')) ]
        return eq_df[eq_df['name'] == symbol]
    elif exch_seg == 'NFO' and ((instrumenttype == 'FUTSTK') or (instrumenttype == 'FUTIDX')):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol)].sort_values(by=['expiry'])
    elif exch_seg == 'NFO' and (instrumenttype == 'OPTSTK' or instrumenttype == 'OPTIDX'):
        return df[(df['exch_seg'] == 'NFO') & (df['instrumenttype'] == instrumenttype) & (df['name'] == symbol) & (df['strike'] == strike_price) & (df['symbol'].str.endswith(pe_ce))].sort_values(by=['expiry'])


## BUY and SELL Indicatore  

In [ ]:
def calculate_inidcator(res_json):
    columns = ['timestamp', 'O', 'H', 'L', 'C', 'V']
    df = pd.DataFrame(res_json['data'], columns=columns)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S')
    
    #Inidcator for Stock 
    df['sup'] = ta.supertrend(high=df['H'], low=df['L'], close=df['C'], period=5, multiplier=1)['SUPERT_7_1.0']
    df['EMA200'] = EMA(df.C, timeperiod=200)
    df['RSI'] = RSI(df.C, timeperiod=7)
    df['ATR'] = ATR(df.H, df.L, df.C, timeperiod=7)

    df['supertrend_Buy'] = df['supertrend_Sell'] = df['EMA3_BUY'] = df['EMA3_SELL'] = df['EMA200_BUY'] = df['EMA200_SELL'] = df['RSI_UP'] = df['RSI_DOWN'] = df['Buy_Signal'] = df['Sell_Signal'] =  0
    df = df.round(decimals=2)

    for i in range(7, len(df)):   
        #SuperTrend Calcualtion 
        if df['C'][i-1]<= df['sup'][i-1] and df['C'][i] > df['sup'][i]:
            df['supertrend_Buy'][i] = 1
        if df['C'][i-1] >= df['sup'][i-1] and df['C'][i] < df['sup'][i]:
            df['supertrend_Sell'][i] = 1
        #EMA-200 Calculator   
        if df['C'][i-1] > df['EMA200'][i]:
            df['EMA200_BUY'][i] = 1
        if df['C'][i-1] < df['EMA200'][i]:
            df['EMA200_SELL'][i] = 1
        #RSI Calculator        
        if df['RSI'][i] >= 60: 
            df['RSI_UP'][i] = 1
        if df['RSI'][i] <= 40: 
            df['RSI_DOWN'][i] = 1
        #BUY and SELL Signal 
        if df['supertrend_Buy'][i] and df['RSI_UP'][i] and df['EMA200_BUY'][i] :
            df['Buy_Signal'][i] = 1
        if df['supertrend_Sell'][i] and df['RSI_DOWN'][i] and df['EMA200_SELL'][i] :
            df['Sell_Signal'][i] = 1

    #Buy and SELL Input 
    df[(df['Buy_Signal'] > 0) | (df['Sell_Signal'] > 0)]
    print(df.tail(10))
    return df

## Getting Historical Data 

In [ ]:
def getHistoricalAPI(token,interval= 'FIVE_MINUTE'):
    to_date= datetime.now()
    from_date = to_date - timedelta(days=5)
    from_date_format = from_date.strftime("%Y-%m-%d %H:%M")
    to_date_format = to_date.strftime("%Y-%m-%d %H:%M")
    try:
        historicParam={
        "exchange": "NSE",
        "symboltoken": token,
        "interval": interval,
        "fromdate": from_date_format, 
        "todate": to_date_format
        }
        candel_json  = credentials.SMART_API_OBJ.getCandleData(historicParam)
        return calculate_inidcator(candel_json)
    except Exception as e:
        print("Historic Api failed: {}".format(e))


## Price, Target, Stoploss Calculation and Order Plcaement 

In [ ]:
def checkSingnal():
    start = time()
    global TRADED_SYMBOL
    
    for symbol in SYMBOL_LIST :
        if symbol not in TRADED_SYMBOL:
            tokenInfo = getTokenInfo(symbol).iloc[0]
            token = tokenInfo['token']
            symbol = tokenInfo['symbol']
            print(symbol, token)
            candel_df = getHistoricalAPI(token)
            if candel_df is not None :
                latest_candel = candel_df.iloc[-1]
                if latest_candel['Buy_Signal'] == 1 :
                   
                    ltp = latest_candel['C']
                    percentage =  0.5
                    #SL_cal = ltp - 5 * percentage
                    #SL = 2 * round(SL_cal / 2)
                    target = ltp + 5 *latest_candel['ATR']
                    target = round(target,1)
                    qty = 79   #qunatity to trade
                    
                    res1= place_order(token,symbol,qty,'BUY','MARKET',0) #buy order
                    #res2 = place_order(token,symbol,qty,'SELL','STOPLOSS_MARKET',0,variety='STOPLOSS',triggerprice= SL) #SL order
                    res3 = place_order(token,symbol,qty,'SELL','LIMIT',target) #taget order
                    print(res1 , res3)
                    print(f'Order Placed for {symbol}  TGT {target} QTY {qty} at {datetime.now()}')
                    TRADED_SYMBOL.append(symbol)

                if latest_candel['Sell_Signal'] == 1:
                    ltp = latest_candel['C']
                    percentage = 0.5
                    #SL_cal = ltp + 5 * percentage
                    #SL = 2 * round(SL_cal / 2)
                    target = ltp - 5 *latest_candel['ATR']
                    target = round(target,1)
                    qty = 79  # qunatity to trade

                    res1 = place_order(token, symbol, qty, 'SELL', 'MARKET', 0)  # buy order
                    #res2 = place_order(token, symbol, qty, 'BUY', 'STOPLOSS_MARKET', 0,variety='STOPLOSS',triggerprice=SL)  # SL order
                    res3 = place_order(token, symbol, qty, 'BUY', 'LIMIT', target)  # taget order
                    print(res1, res3)
                    print(f'Order Placed for {symbol}  TGT {target} QTY {qty} at {datetime.now()}')
                    TRADED_SYMBOL.append(symbol)


    interval = timeFrame - (time()-start)   
    print(interval)
    threading.Timer(interval, checkSingnal).start()

## Every Interval Code Re-exceute 

In [ ]:
if __name__ == '__main__':
    intializeSymbolTokenMap()
    obj=SmartConnect(api_key=credentials.API_KEY)
    data = obj.generateSession(credentials.USER_NAME,credentials.PWD)
    credentials.SMART_API_OBJ = obj
   
    interval = timeFrame - datetime.now().second
    print(f"Code run after {interval} sec")
    sleep(interval)
    checkSingnal()